In [204]:
import pandas as pd
import s3fs
import folium


In [205]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [206]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [207]:
df = pd.read_csv('post_processed_listings.csv')

In [208]:
df.head()

,id,Type,Nb_chambre,loc,surface,prix
0,apimo-85473173,Appartement,1,Châtelet,29,364000
1,hektor-Les-4-Quartiers-383,Appartement,1,Châtelet,23,301000
2,hektor-PARISLUXURYHOMES-4007,Appartement,5,Palais Royal,106,2490000
3,ag754594-448898324,Studio,1,Saint-Germain,27,365000
4,keller-williams-1-34_1_53-180645,Appartement,2,Vendôme,67,1190000


In [209]:
df.shape

(8830, 6)

In [210]:
# Ajouter ", Paris" uniquement si "Paris" n'est pas déjà présent dans la colonne loc
df['loc'] = df['loc'].apply(lambda x: x if 'Paris' in x else x + ", Paris")

# Vérification
df.head()


,id,Type,Nb_chambre,loc,surface,prix
0,apimo-85473173,Appartement,1,"Châtelet, Paris",29,364000
1,hektor-Les-4-Quartiers-383,Appartement,1,"Châtelet, Paris",23,301000
2,hektor-PARISLUXURYHOMES-4007,Appartement,5,"Palais Royal, Paris",106,2490000
3,ag754594-448898324,Studio,1,"Saint-Germain, Paris",27,365000
4,keller-williams-1-34_1_53-180645,Appartement,2,"Vendôme, Paris",67,1190000


In [211]:
## Reprise du code de MAXIME pour avoir les même notations ##

# Uniformiser le format des localisation
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)
df_ = df.copy()
df_[['parentheses']] = df_['loc'].str.extract(r'\((.*?)\)')
df_['concat'] = df_['parentheses'].fillna('')
df_['loc'] = df_['concat'].where(df_['parentheses'].notna(), df_['loc'])
df_.drop(columns=['parentheses', 'concat'], inplace=True)

df_[['parentheses']] = df_['loc'].str.extract(r'^(.*?)\s*\s-')
df_['concat'] = df_['parentheses'].fillna('')
df_['loc'] = df_['concat'].where(df_['parentheses'].notna(), df_['loc'])
df_.drop(columns=['parentheses', 'concat'], inplace=True)

df_zones=df_["loc"].drop_duplicates().copy()
df=df_.copy()

In [212]:
from geopy.geocoders import Nominatim
from functools import partial
geolocator = Nominatim(user_agent="testing")
geocode = partial(geolocator.geocode, language="fr")
geocode_with_paris = lambda query: geolocator.geocode("%s, Paris FR" % query)

In [213]:
# Initialiser une liste vide pour stocker les zones
tableau_zones = []

# Parcourir les zones et les ajouter au tableau
for zone in df_zones:
    print(zone)  # Affiche chaque zone
    tableau_zones.append(zone)  # Ajoute chaque zone au tableau

# Afficher le tableau final après la boucle
print("\nTableau des zones :")
print(tableau_zones)

Châtelet, Paris
Palais Royal, Paris
Saint-Germain, Paris
Vendôme, Paris
Gaillon, Paris
Saint-Thomas d'Aquin, Paris
75001 Paris 1er
Sentier, Paris
Montorgueil, Paris
75002 Paris 2e
Arts-et-Métiers, Paris
Sainte-Avoye, Paris
Archives, Paris
75003 Paris 3e
Enfants Rouges, Paris
République, Paris
Saint-Merri, Paris
Arsenal, Paris
75004 Paris 4e
Les Iles, Paris
Saint-Gervais, Paris
75000 Paris
Monge, Paris
Jardin des Plantes, Paris
Val de Grâce, Paris
Sorbonne, Paris
Rennes, Paris
Montparnasse, Paris
Saint-Germain des Près, Paris
Odéon, Paris
Saint-Placide, Paris
Notre Dame des Champs, Paris
75006 Paris 6e
Saint-Michel, Paris
Hoche Friedland, Paris
Elysée, Paris
Monceau, Paris
Triangle d'Or, Paris
Europe, Paris
Saint-Philippe du Roule, Paris
Mairie, Paris
75008 Paris 8e
Clichy, Paris
Grands Boulevards, Paris
Lorette, Paris
Opéra, Paris
Trudaine, Paris
Porte Saint-Martin, Paris
75009 Paris 9e
Saint-Vincent de Paul, Paris
Château d'Eau, Paris
Goncourt, Paris
Canal Saint-Martin, Paris
Louis Bl

In [214]:
len(tableau_zones)

140

In [215]:
from geopy.geocoders import Nominatim
import time

# Créer un objet géocodeur Nominatim pour extraire la longitude et la latitude de chaque zone
geolocator = Nominatim(user_agent="my_geocoder")

# Fonction pour obtenir les coordonnées géographiques (longitude et latitude) d'une adresse
def obtenir_coordonnees(tableau_zones):
    try:
        location = geolocator.geocode(tableau_zones, timeout=5)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur pour {tableau_zones}: {e}")
        return None, None


# Créer un dictionnaire pour stocker les adresses et leurs coordonnées géographiques
coordonnees_dict = {}

for zone in tableau_zones:
    lat, lon = obtenir_coordonnees(zone)  # Passer 'zone_paris' à la fonction
    coordonnees_dict[zone] = (lat, lon)   # Utiliser 'zone_paris' comme clé
    
    # Pause pour éviter un blocage par le serveur
    time.sleep(1)

# Créer une liste pour stocker les informations des adresses
coordonnees = []

# Afficher et stocker les coordonnéesde toutes les zones
print("\nCoordonnées des 3 premières adresses :")
for zone, coord in coordonnees_dict.items():
    print(f"Adresse: {zone}, Latitude: {coord[0]}, Longitude: {coord[1]}")
    coordonnees.append({"adresse": zone, "latitude": coord[0], "longitude": coord[1]})  # Stocker les données

# Afficher le contenu de coordonnees
print("\nContenu de coordonnees2 :")
for item in coordonnees:
    print(item)


Coordonnées des 3 premières adresses :
Adresse: Châtelet, Paris, Latitude: 48.8568781, Longitude: 2.3483592
Adresse: Palais Royal, Paris, Latitude: 48.863584700000004, Longitude: 2.3362042200938715
Adresse: Saint-Germain, Paris, Latitude: 48.85655255, Longitude: 2.333331104782813
Adresse: Vendôme, Paris, Latitude: 48.86690005, Longitude: 2.3285489140926545
Adresse: Gaillon, Paris, Latitude: 48.869135150000005, Longitude: 2.332908770335507
Adresse: Saint-Thomas d'Aquin, Paris, Latitude: 48.85424375, Longitude: 2.3268829064838443
Adresse: 75001 Paris 1er, Latitude: 48.8646144, Longitude: 2.334396
Adresse: Sentier, Paris, Latitude: 48.867518, Longitude: 2.3465656
Adresse: Montorgueil, Paris, Latitude: 48.8641678, Longitude: 2.3467038
Adresse: 75002 Paris 2e, Latitude: 48.868743, Longitude: 2.341688
Adresse: Arts-et-Métiers, Paris, Latitude: 48.8654718, Longitude: 2.3561116
Adresse: Sainte-Avoye, Paris, Latitude: 48.862699750000004, Longitude: 2.354135471358302
Adresse: Archives, Paris, L

In [216]:
coordonnees

[{'adresse': 'Châtelet, Paris',
  'latitude': 48.8568781,
  'longitude': 2.3483592},
 {'adresse': 'Palais Royal, Paris',
  'latitude': 48.863584700000004,
  'longitude': 2.3362042200938715},
 {'adresse': 'Saint-Germain, Paris',
  'latitude': 48.85655255,
  'longitude': 2.333331104782813},
 {'adresse': 'Vendôme, Paris',
  'latitude': 48.86690005,
  'longitude': 2.3285489140926545},
 {'adresse': 'Gaillon, Paris',
  'latitude': 48.869135150000005,
  'longitude': 2.332908770335507},
 {'adresse': "Saint-Thomas d'Aquin, Paris",
  'latitude': 48.85424375,
  'longitude': 2.3268829064838443},
 {'adresse': '75001 Paris 1er', 'latitude': 48.8646144, 'longitude': 2.334396},
 {'adresse': 'Sentier, Paris', 'latitude': 48.867518, 'longitude': 2.3465656},
 {'adresse': 'Montorgueil, Paris',
  'latitude': 48.8641678,
  'longitude': 2.3467038},
 {'adresse': '75002 Paris 2e', 'latitude': 48.868743, 'longitude': 2.341688},
 {'adresse': 'Arts-et-Métiers, Paris',
  'latitude': 48.8654718,
  'longitude': 2.35

In [219]:
len(coordonnees)

140

In [221]:
# Diviser les adresses en fonction des noms spécifiques ou vagues
noms_specifiques = []
noms_vagues = []

for item in coordonnees:
    if any(char.isdigit() for char in item['adresse']):  # Vérifier si l'adresse contient des chiffres (arrondissements)
        noms_vagues.append(item)
    else:
        noms_specifiques.append(item)

print(noms_specifiques)
print(noms_vagues)

[{'adresse': 'Châtelet, Paris', 'latitude': 48.8568781, 'longitude': 2.3483592}, {'adresse': 'Palais Royal, Paris', 'latitude': 48.863584700000004, 'longitude': 2.3362042200938715}, {'adresse': 'Saint-Germain, Paris', 'latitude': 48.85655255, 'longitude': 2.333331104782813}, {'adresse': 'Vendôme, Paris', 'latitude': 48.86690005, 'longitude': 2.3285489140926545}, {'adresse': 'Gaillon, Paris', 'latitude': 48.869135150000005, 'longitude': 2.332908770335507}, {'adresse': "Saint-Thomas d'Aquin, Paris", 'latitude': 48.85424375, 'longitude': 2.3268829064838443}, {'adresse': 'Sentier, Paris', 'latitude': 48.867518, 'longitude': 2.3465656}, {'adresse': 'Montorgueil, Paris', 'latitude': 48.8641678, 'longitude': 2.3467038}, {'adresse': 'Arts-et-Métiers, Paris', 'latitude': 48.8654718, 'longitude': 2.3561116}, {'adresse': 'Sainte-Avoye, Paris', 'latitude': 48.862699750000004, 'longitude': 2.354135471358302}, {'adresse': 'Archives, Paris', 'latitude': 48.8596256, 'longitude': 2.362541376680502}, {'

In [222]:
### CREATION DE LA CARTE ###

# Initialiser la carte centrée sur Paris
carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajouter des points sur la carte pour les adresses avec coordonnées
for lieu in noms_specifiques :          # J'affiche seulement les adresses spécifiques (pas les vagues pour eviter tout problème parce qu'elles n'apparaissent pas au bond endroit sur la carte...)
    if lieu["latitude"] is not None and lieu["longitude"] is not None:
        folium.Marker(
            location=[lieu["latitude"], lieu["longitude"]],
            popup=lieu["adresse"],      # Texte affiché lorsqu'on clique sur le point
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(carte)

# Sauvegarder la carte dans un fichier HTML
carte.save("carte_paris_points.html")
print("Carte créée : carte_paris_points.html")

Carte créée : carte_paris_points.html


In [223]:
## Affichage de la carte
carte

In [244]:
# Regrouper par 'loc' et calculer la moyenne des prix, des surfaces et le nombre d'annonces
resultat = df.groupby('loc', as_index=False).agg(
    moyenne_prix=('prix', 'mean'),      # Moyenne des prix
    moyenne_surface=('surface', 'mean'),  # Moyenne des surfaces
    nombre_annonces=('prix', 'size')   # Nombre d'annonces (compte les lignes)
)

# Calculer la moyenne des prix au m²
resultat['prix_m2'] = resultat['moyenne_prix'] / resultat['moyenne_surface']

# Supprimer les colonnes intermédiaires si elles ne sont pas nécessaires
resultat = resultat[['loc', 'prix_m2', 'nombre_annonces']]  # Garder uniquement loc, prix_m2 et nombre d'annonces

# Vérification des résultats
print(resultat)

                         loc       prix_m2  nombre_annonces
0                75000 Paris   4456.521739                1
1            75001 Paris 1er  20959.770115                3
2             75002 Paris 2e  12623.655914                3
3             75003 Paris 3e  10774.245115                6
4             75004 Paris 4e  21924.863388                3
..                       ...           ...              ...
135      Val de Grâce, Paris  13455.888651                9
136  Vallée de Fecamp, Paris   8803.143394               98
137         Vaugirard, Paris  10369.205115              206
138           Vendôme, Paris  17642.301545               19
139            Violet, Paris  11067.360796               53

[140 rows x 3 columns]


In [245]:
# Convertir les coordonnées en DataFrame
coord_df = pd.DataFrame(coordonnees)

# Fusionner les deux DataFrames
resultat = resultat.merge(coord_df[['adresse', 'latitude', 'longitude']], 
                          left_on='loc', 
                          right_on='adresse', 
                          how='left')

# Supprimer la colonne 'adresse' qui est redondante après la fusion
resultat.drop(columns=['adresse'], inplace=True)

# Vérification des résultats
print(resultat)

                         loc       prix_m2  nombre_annonces   latitude  \
0                75000 Paris   4456.521739                1  48.858890   
1            75001 Paris 1er  20959.770115                3  48.864614   
2             75002 Paris 2e  12623.655914                3  48.868743   
3             75003 Paris 3e  10774.245115                6  48.864212   
4             75004 Paris 4e  21924.863388                3  48.856202   
..                       ...           ...              ...        ...   
135      Val de Grâce, Paris  13455.888651                9  48.842243   
136  Vallée de Fecamp, Paris   8803.143394               98  48.834826   
137         Vaugirard, Paris  10369.205115              206  48.839587   
138           Vendôme, Paris  17642.301545               19  48.866900   
139            Violet, Paris  11067.360796               53  48.844960   

     longitude  
0     2.320041  
1     2.334396  
2     2.341688  
3     2.360936  
4     2.355619  
..       

In [246]:
# Supprimer les lignes contenant des NaN dans le DataFrame 'resultat'
resultat = resultat.dropna()

# Vérification des résultats
print("DataFrame après suppression des NaN :")
print(resultat)

DataFrame après suppression des NaN :
                         loc       prix_m2  nombre_annonces   latitude  \
0                75000 Paris   4456.521739                1  48.858890   
1            75001 Paris 1er  20959.770115                3  48.864614   
2             75002 Paris 2e  12623.655914                3  48.868743   
3             75003 Paris 3e  10774.245115                6  48.864212   
4             75004 Paris 4e  21924.863388                3  48.856202   
..                       ...           ...              ...        ...   
135      Val de Grâce, Paris  13455.888651                9  48.842243   
136  Vallée de Fecamp, Paris   8803.143394               98  48.834826   
137         Vaugirard, Paris  10369.205115              206  48.839587   
138           Vendôme, Paris  17642.301545               19  48.866900   
139            Violet, Paris  11067.360796               53  48.844960   

     longitude  
0     2.320041  
1     2.334396  
2     2.341688  
3    

In [248]:
# Normaliser les tailles des points en fonction du nombre d'annonces
min_radius = 5
max_radius = 20

def scale_radius(nombre_annonces):
    return min_radius + (max_radius - min_radius) * (nombre_annonces - resultat['nombre_annonces'].min()) / (resultat['nombre_annonces'].max() - resultat['nombre_annonces'].min())

# Création de la carte
carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajout des points
for _, row in resultat.iterrows():
    couleur = prix_m2_to_color(row['prix_m2'])  # Couleur basée sur le prix/m²
    radius = scale_radius(row['nombre_annonces'])  # Taille basée sur le nombre d'annonces
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,  # Taille des points
        color='black',  # Contour noir
        fill=True,
        fill_color=couleur,  # Remplissage rouge
        fill_opacity=0.9,
        popup=f"{row['loc']}<br>Prix/m² : {row['prix_m2']:.2f} €<br>Nombre d'annonces : {row['nombre_annonces']}"
    ).add_to(carte)

# Sauvegarder la carte
carte.save("carte_prix_m2_taille.html")
print("Carte créée : carte_prix_m2_taille.html")

Carte créée : carte_prix_m2_taille.html


In [249]:
# Les prix bas : en rouge clair.
# Les prix élevés : en rouge foncé.

carte